In [15]:
import numpy as np
import tensorflow as tf
from collections import defaultdict

In [16]:
def load_ferdosi_for_wrod2vec(dir):
    sentences = []
    with open(dir, 'r') as f:
        for b in f:
            m1, m2 = b.split(",")
            split = m1.split(" ")
            sentences.append([elem for elem in split if elem!=''])
            split = m2.split(" ")
            split[-1] = split[-1][:len(split[-1])-1]
            sentences.append([elem for elem in split if elem!=''])
        return np.array(sentences)

In [17]:
dataset_dir = '/Users/Alireza/Desktop/Current Semester/Deep Learning/Assignments/DL_HW4/ferdosi.txt'
sentences = load_ferdosi_for_wrod2vec(dataset_dir)
print (sentences[0])
print (sentences[1])

['به', 'نام', 'خداوند', 'جان', 'و', 'خرد']
['کزین', 'برتر', 'اندیشه', 'برنگذرد']


In [18]:
window_size = 5
hidden_layer = 25
epochs = 50
learning_rate = 0.01

In [19]:
def word2onehot(word, v_count, word_index):
    word_vec = np.zeros(v_count)
    word_index = word_index[word]
    word_vec[word_index] = 1
    return word_vec
def softmax(x): 
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum()

In [20]:
def generate_training_data(sentences):
    word_counts = defaultdict(int)
    for row in sentences:
        for word in row:
            word_counts[word] += 1
    v_count = len(word_counts.keys())
    words_list = list(word_counts.keys())
    word_index = dict((word, i) for i, word in enumerate(words_list))
    index_word = dict((i, word) for i, word in enumerate(words_list))

    training_data = np.zeros((v_count, v_count))
    print (training_data.shape)
    for sentence in sentences:
        sent_len = len(sentence)
        for i, word in enumerate(sentence):
            for j in range(max(i - window_size, 0), i + window_size + 1):
                if j != i and j <= sent_len-1 and j >= 0 and word_index[word] != word_index[sentence[j]]:
                    training_data[word_index[word], word_index[sentence[j]]] += 1
    return training_data, words_list, word_index, index_word

In [21]:
training_data, words_list, word_index, index_word = generate_training_data(sentences)

(17658, 17658)


In [22]:
print (training_data[0])
print (index_word[0])
print (index_word[1])

[  0. 160.  25. ...   1.   0.   0.]
به
نام


In [26]:
# loss of this implementation is too big!
class word2vec(object): 
    def __init__(self): 
        self.words = []
        self.V = None
        self.word_index = None
        self.lr = learning_rate
       
    def feed_forward(self,X_index): 
        self.h = self.W[X_index].reshape(hidden_layer,1)
#         print (self.h.shape)
        self.u = np.dot(self.W1.T,self.h).reshape(self.V,1)
#         print (self.u.shape)
        self.y = softmax(self.u).reshape(self.V,1)
#         print (self.y.shape)
        return self.y 
           
    def backpropagate(self,x_index,t): 
#         print("t shape = " + str(t.shape))
        e = self.y - np.asarray(t).reshape(self.V,1)
#         print("e shape = " + str(e.shape))
        dLdW1 = np.dot(self.h,e.T) 
#         print(np.dot(self.W1,e).shape)
        dLdW = np.zeros((self.V, hidden_layer)) 
        dLdW[x_index] = np.dot(self.W1,e).flatten()
#         print(dLdW.shape)
        self.W1 = self.W1 - learning_rate*dLdW1 
        self.W = self.W - learning_rate*dLdW 
           
    def train(self, sentences, epochs): 
        word_counts = defaultdict(int)
        for row in sentences:
            for word in row:
                word_counts[word] += 1
        self.V = len(word_counts.keys())
        self.words = list(word_counts.keys())
        self.word_index = dict((word, i) for i, word in enumerate(self.words))
                
        self.W = np.random.normal(0, 0.0001, (self.V, hidden_layer)) 
        self.W1 = np.random.normal(0, 0.0001, (hidden_layer, self.V))
        print (sentences.shape)
        for x in range(1,epochs):         
            self.loss = 0
            for w_target_index, w_context in enumerate(training_data):
                self.feed_forward(w_target_index) 
                self.backpropagate(w_target_index, w_context) 
                C = 0
                for m in range(self.V): 
                    if(w_context[m]): 
                        self.loss += -1*self.u[m][0] 
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u))) 
            print("epoch ",x, " loss = ",self.loss) 
            self.lr *= 1/( (1+self.lr*x) ) 
              
    def predict(self,word,number_of_predictions): 
        if word in self.words:
            X = word2onehot(word, self.V, self.word_index)
            prediction = self.feed_forward(X) 
            output = {} 
            for i in range(self.V): 
                output[prediction[i][0]] = i 
               
            top_context_words = [] 
            for k in sorted(output,reverse=True): 
                top_context_words.append(self.words[output[k]]) 
                if(len(top_context_words)>=number_of_predictions): 
                    break
       
            return top_context_words 
        else: 
            print("Word not found in dicitonary") 

In [27]:
w2v = word2vec()
w2v.train(sentences, epochs)

(99218,)
epoch  1  loss =  8630465.893000007
epoch  2  loss =  17405388.29755063


/Users/alireza/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: overflow encountered in exp
/Users/alireza/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


KeyboardInterrupt: 

In [52]:
def make_batches(x, batch_size):
    n = len(x)
    steps = n // batch_size
    if n % batch_size != 0:
        steps += 1
    x_batches = np.array_split(x, steps)
    return x_batches

def generate_training_data(sentences, word_index, dictionary_size):
    x = []
    y = []
    for sentence in sentences:
        sent_len = len(sentence)
        for i, word in enumerate(sentence):
            w_target = np.zeros(v_count)
            w_target[word_index[word]] = 1

            w_context = np.zeros(v_count)
            for j in range(i - window_size, i + window_size + 1):
                if j != i and j <= sent_len-1 and j >= 0:
                    w_context[word_index[sentence[j]]] += 1
        x.append(w_target)
        y.append(w_context)
    return x, y

In [47]:
dictionary_size = len(words_list)
# Inputs
X = tf.placeholder("float", shape=[None, dictionary_size])
y = tf.placeholder("float", shape=[None, dictionary_size])

# Dictionary of Weights and Biases
weights = {
  'W1': tf.Variable(tf.random_normal([dictionary_size, hidden_layer])),
  'W2': tf.Variable(tf.random_normal([hidden_layer, dictionary_size])),
}

biases = {
  'b1': tf.Variable(tf.random_normal([hidden_layer])),
  'b2': tf.Variable(tf.random_normal([dictionary_size])),
}

In [48]:
def forward_propagation(x):
    hidden_1 = tf.add(tf.matmul(x, weights['W1']), biases['b1'])   
    out_layer = tf.add(tf.matmul(hidden_1, weights['W2']), biases['b2'])
    
    softmax_out = tf.nn.softmax(out_layer)    
    return softmax_out

In [58]:
yhat = forward_propagation(X)
ypredict = tf.argmax(yhat, axis=1)

In [61]:
learning_rate = 0.1
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=yhat))
loss = tf.reduce_mean(tf.log(SigPositive))
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

In [62]:
# Initializing the variables
init = tf.global_variables_initializer()

batch_size = 128
sentences_batches = make_batches(sentences, batch_size)
N_Batches = len(sentences_batches)

epochs = 100

sess = tf.Session()
sess.run(init)

train_loss = np.zeros(epochs)

#EPOCHS
for epoch in range(epochs):
    
    #Stochasting Gradient Descent
    batch_loss = np.zeros(N_Batches)
    for i in range(N_Batches):
        X_train, Y_train = generate_training_data(sentences_batches[i], word_index, dictionary_size)
        _, batch_loss[i] = sess.run([train_op, cost], feed_dict={X: X_train, 
                                                                 y: Y_train})
        if i % 100 == 0:
            print("Batch %3d/%d: Loss:%0.4f"%(i + 1, N_Batches, batch_loss[i]))
        
    train_loss[epoch] = np.mean(batch_loss)
    print("Epoch = %d, loss = %.2f%" % (epoch + 1, train_loss[epoch]))

Batch   1/776: Loss:43.5472
Batch 101/776: Loss:43.7140
Batch 201/776: Loss:43.3476
Batch 301/776: Loss:43.3867


KeyboardInterrupt: 